In [ ]:
import random
import math
import pandas as pd
import numpy.random as npr

In [ ]:
lv = []
with open("/content/xql662.tsp") as f:
  for i in f:
    lv += [[int(j) for j in i[:-1].split()]]

dict_cities = {}
for i in lv:
  dict_cities[i[0]] = [i[1],i[2]]

In [ ]:
def factory(count_cities):
  paths = [i+1 for i in range(count_cities)]
  #shuffle_paths = paths[1:]
  #random.shuffle(shuffle_paths)
  #paths[1:] = shuffle_paths
  return paths 

In [ ]:
def crossover(p1, p2):
    p1 = p1[1:]
    p2= p2[1:]
    #order crossover
    children = [-1 for i in range(len(p1))]
    random_1 = random.randrange(len(p1)) 
    random_2 =  random.randrange(len(p1))
    if random_2 < random_1 :
      random_1 , random_2 = random_2, random_1
    children[random_1 : random_2] = p1[random_1 : random_2]
    #p2 = p2[random_2:] + p2[:random_2]
    for i in p1[random_1 : random_2]:
      p2.remove(i)
    for i in range(len(children[random_2:])):
      children[random_2+i] = p2[0]
      del p2[0]
    for i in range(len(children[:random_1])):
      children[i] = p2[0]
      del p2[0]
    return [1] + children

In [ ]:
def fintess_function(path, dict_cities):
  path = path + [1]
  dist = 0
  for n, i in enumerate(path[1:]):
   #print(dict_cities[path[n]])
    #print(dict_cities[path[n]][0], dict_cities[path[n]][1])
    dist += math.sqrt( (dict_cities[i][0] - dict_cities[path[n]][0])**2 + (dict_cities[i][1]- dict_cities[path[n]][1])**2 )
    #print("___")
  return dist

In [ ]:
def selection(population, populations_size):
  df = pd.DataFrame({"individ":population, "score": [fintess_function(i, dict_cities) for i in population]})
  df = df.sort_values("score")
  #print(df[:populations_size])
  return list(df["individ"][:populations_size]), min(list(df["score"][:populations_size]))

In [ ]:
def mutation(individs, SWAP_MUTATION, INSERT_MUTATION, SCRAMBLE_MUTATION, INVERSION_MUTATION):

  # swap mutation
  for j in range(SWAP_MUTATION):
      i = individs[j]
      rand = random.randrange(1, len(individs))
      random_1 = random.randrange(1, len(i))
      random_2 =  random.randrange(1, len(i))
      i[random_1], i[random_2] = i[random_2],i[ random_1]
  # insert mutation
  for j in range(INSERT_MUTATION):
      i = individs[j]
      random_1 = random.randrange(1, len(i)-1)
      random_2 =  random.randrange(1, len(i))
      i[random_1+1], i[random_2] = i[random_2],i[ random_1+1]
  # scramble mutation
  for j in range(SCRAMBLE_MUTATION):
      i = individs[j]
      random_1 = random.randrange(1, len(i)-1)
      random_2 =  random.randrange(1, len(i))
      shuffle_list = i[random_1:random_2]
      random.shuffle(shuffle_list)
      i[random_1:random_2] = shuffle_list
  # inversion mutation
  for j in range(INVERSION_MUTATION):
      i = individs[j]
      if random_2 < random_1 :
        random_1 , random_2 = random_2, random_1
      random_1 = random.randrange(1, len(i))
      random_2 =  random.randrange(1, len(i))
      invers_list = i[random_1:random_2][::1]
      i[random_1:random_2] = invers_list

  return individs

In [ ]:
def selectOne(population):
    max = sum(1/population.score)
    selection_probs = [(1/c)/max for c in df.score]
    return df["individ"][npr.choice(len(population), p=selection_probs)]

In [ ]:
GENERATIONS = 100000
POPULATIONS_SIZE = 1500
COUNT_CITIES = 662
PRECENT_CROSSOVER = 0.9
best_ever = 10000000
ELETISM = 200
SWAP_MUTATION = 150
INSERT_MUTATION = 150
SCRAMBLE_MUTATION = 30
INVERSION_MUTATION = 30

population = []
for i in range(GENERATIONS):
  #factor
  if population == []:
    for j in range(POPULATIONS_SIZE):
      population += [factory(COUNT_CITIES)]

  # elitism
  new_population = []
  df = pd.DataFrame({"individ":population, "score": [fintess_function(i, dict_cities) for i in population]})
  df = df.sort_values("score")
  base = list(df["individ"][:ELETISM])

  # crossover
  for j in range(round(PRECENT_CROSSOVER*round(POPULATIONS_SIZE/2))):
    new_population += [crossover(selectOne(df),selectOne(df))]
  population += new_population

  #mutation
  random.shuffle(population)
  population = mutation(population, SWAP_MUTATION, INSERT_MUTATION, SCRAMBLE_MUTATION, INVERSION_MUTATION)
  
  #selection
  population, best_score = selection(population, POPULATIONS_SIZE- ELETISM)
  population = base + population

  if best_score < best_ever:
    best_ever = best_score

  print(f"Generation #{i}, best_score_ever = {best_ever}, best_current_score = {best_score}")


  


  